In [ ]:
import requests as req
import qgrid
import pandas as pd
from IPython.display import display, HTML

BOOTSTRAP_STATIC_URL = 'https://fantasy.premierleague.com/api/bootstrap-static/'
ELEMENT_DETAIL = 'https://fantasy.premierleague.com/api/element-summary/{ELEMENT_ID}'
ELEMENTS_LIVE = 'https://fantasy.premierleague.com/api/event/{GAMEWEEK}/live/'
ENTRY_DETAIL = 'https://fantasy.premierleague.com/api/entry/{USER_ID}/'
GAMEWEEK_PICKS = 'https://fantasy.premierleague.com/api/entry/{USER_ID}/event/{GAMEWEEK}/picks'
CLASSIC_LEAGUE_STANDINGS = 'https://fantasy.premierleague.com/api/leagues-classic-standings/{LEAGUE_ID}?page_new_entries=1&page_standings=1&phase=1'
H2H_LEAGUE_STANDINGS = ''
DREAM_TEAM = 'https://fantasy.premierleague.com/api/dream-team/{GAMEWEEK}/'

DRAFT_BOOTSTRAP_STATIC_URL = 'https://draft.premierleague.com/api/bootstrap-static'
DRAFT_LEAGUE_DETAILS = 'https://draft.premierleague.com/api/league/{LEAGUE_ID}/details'
DRAFT_USER_PICK = 'https://draft.premierleague.com/api/entry/{USER_ID}/event/{GAMEWEEK}'

# current user info
USER_ID = '653325'
DF_LEAGUE_ID = '80782'

In [ ]:
resp = req.get(BOOTSTRAP_STATIC_URL)
BOOTSTRAP_STATIC = resp.json()
# store bootstrap static and live points to redis/postgres? even pandas will do

CURRENT_GW = list(filter(lambda event: event['is_current'] == True, BOOTSTRAP_STATIC['events']))
if CURRENT_GW:
    CURRENT_GW = CURRENT_GW[0]
NEXT_GW = list(filter(lambda event: event['is_next'] == True, BOOTSTRAP_STATIC['events']))
if NEXT_GW:
    NEXT_GW = NEXT_GW[0]
    
# get live points
# query with current GW to the 

In [ ]:
# user info
resp = req.get(ENTRY_DETAIL.format(USER_ID=USER_ID))
USER_INFO = resp.json()

# gather league list
CLASSIC_LEAGUES = USER_INFO['leagues']['classic']
H2H_LEAGUES = USER_INFO['leagues']['h2h']

# mini league standings - need auth for this

# construct greq for classic leagues

url_keys = ''

urls = [
    
]

In [ ]:
resp = req.get(DRAFT_BOOTSTRAP_STATIC_URL)
DFT_BOOTSTRAP_STATIC = resp.json()
# store bootstrap static and live points to redis/postgres? even pandas will do

DFT_CURRENT_GW = DFT_BOOTSTRAP_STATIC['events']['current']
DFT_NEXT_GW = DFT_BOOTSTRAP_STATIC['events']['next']

# df for all lists

DFT_ELEMENTS_DF = pd.DataFrame(DFT_BOOTSTRAP_STATIC['elements'])
DFT_ELEMENTS_DF.set_index('id', inplace=True)


DFT_ELEMENT_TYPES = pd.DataFrame(DFT_BOOTSTRAP_STATIC['element_types'])
DFT_ELEMENT_TYPES.set_index('id', inplace=True)

In [ ]:
color_mapping = {
    'GKP': 'pink',
    'DEF': 'red',
    'MID': 'blue',
    'FWD': 'green'
}

def color_scheme(value):
    return 'color: %s' % color_mapping(value)

In [ ]:
# get all team details
resp = req.get(DRAFT_LEAGUE_DETAILS.format(LEAGUE_ID=DF_LEAGUE_ID))

entries_df = pd.DataFrame(resp.json()['league_entries'])

standings_df = pd.DataFrame(resp.json()['standings'])

In [ ]:
# show waiver pick
qgrid.show_grid(entries_df[['entry_name', 'player_first_name', 'player_last_name', 'waiver_pick']])

In [ ]:
for index, row in entries_df.iterrows():
    # get all players for each team
    'https://draft.premierleague.com/api/entry/{USER_ID}/event/{GAMEWEEK}'
    res = req.get(DRAFT_USER_PICK.format(USER_ID=row['entry_id'], GAMEWEEK=DFT_CURRENT_GW))

    df = pd.DataFrame(columns=['element_type', 'web_name'])
    for item in res.json()['picks']:
        elem = DFT_ELEMENTS_DF.loc[item['element'], :]
        element_iter = {
            'element_type': DFT_ELEMENT_TYPES.loc[elem['element_type'], :]['plural_name_short'], 
            'web_name': elem['web_name']
        }
        df = df.append(element_iter, ignore_index=True)

    display(row['player_first_name'])
    display(df)
    df = df.iloc[0:0]